   # Validação de Dados Modelados com Observados

Carregar arquivos para comparar dados monitorados com séries modelada (arquivos combine)

In [30]:
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
#from mpl_toolkits.basemap import Basemap
import seaborn as sns
import datetime as dt
from bisect import bisect_left
import math
import utm
%matplotlib inline

def takeClosest(myList, myNumber):
    """
    Assumes myList is sorted. Returns closest value to myNumber.
    If two numbers are equally close, return the smallest number.
    """
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before


## *Entradas*
* Selecionar arquivos de origem para preparar figuras.
* Inicializar variáveis para figuras.

### Parametrização dos arquivos de entrada

Ajustar diretório que contem o arquivo da estação e nomes de rodadas.

In [41]:
directory = str('E:/Mestrado/1.Resultados/QualidadedoAr/')
wrfdir = str('E:/Mestrado/1.Resultados/Meteorologia/')
mes   = ['jan','jul']
grade = 'd04_1km'
data  = ['20150109','20150724']
datawrf  = ['01-09','07-24']
periodo = ['Verão', 'Inverno']

modelagens = ['-','_PT_ED-100I'] # Carregando dois cenários base (ED e PT, respectivamente)
modnames = ['ED', 'PT_ED-100I']
#Demais cenários:
#ED = '-20I',           '-20T',            '-20I-20T'
#PT = '_PT-20_ED-100I', '_PT_ED-100I-20T', '_PT-20_ED-100I-20T'
shapes = ['pirashape','RMSP']

spinup = 48
run_len = 24*7
gmt = 3

date_index = 0

In [42]:
#DADOS ESTACAO
estacao     = 'PIRACICABA (A)'
estlat,estlon = utm.to_latlon(easting=227797, northing=7487124, zone_letter='k', zone_number=23)
est = pd.read_csv(directory+'piracicaba.csv', delimiter=';')
#est['Hora'] = est['Hora'].apply(lambda x: '24:00' if x == '24:00:00' else x)
est['Hora'] = est['Hora'].apply(lambda x: dt.timedelta(hours=float(x[:2])))
est['Data'] = est['Data'].apply(lambda x: dt.datetime.strptime(x,'%d/%m/%Y'))
est['Data'] += est['Hora']
est.drop(columns=['Unnamed: 0','Hora','wind_dir','wind_dir_avg','temp','ur','wind_vel'], inplace=True)
est.set_index('Data', inplace=True)

## Abrir arquivos COMBINE ACONC e DEP

In [43]:
aconc = []
#dep=[]
for m in modelagens:
    aconc.append(xr.open_dataset(directory+mes[date_index]+'.'+grade+m+'/COMBINE_ACONC_v521_intel_pira_riz_nudge_'+
                        mes[date_index]+'_'+data[date_index]+'.nc'))
for i in range(0,len(aconc)):
    aconc[i] = aconc[i].isel(LAY=0)

## Abrir arquivo WRF para buscar valores de LAT / LON

In [44]:
nc_wrf = Dataset(wrfdir+'arqWRF/'+mes[0]+'/wrf_fnl_pira_pedruzzi2016_nudge_'+
                 mes[0]+'/wrfout_'+grade[:3]+'_2015-'+datawrf[0]+'_00%3A00%3A00')

## Preparar coordenadas para XArray
* DATAS
* ALTITUDE
* LATITUDE
* LONGITUDE

In [45]:
# DATAS
dates = []
dates = [dt.datetime.strptime(str(aconc[0].attrs['SDATE']),'%Y%j')]
for i in range(1,aconc[0].dims['TSTEP']):
    dates.append(dates[0]+dt.timedelta(hours=i))
    
# LATITUDE
# LONGITUDE
croplat = int((nc_wrf.dimensions['south_north'].size-aconc[0].dims['ROW'])/2)
croplon = int((nc_wrf.dimensions['south_north'].size-aconc[0].dims['ROW'])/2)
lats      = nc_wrf.variables['XLAT'][0,croplat:-croplat,croplat:-croplat]
longs     = nc_wrf.variables['XLONG'][0,croplon:-croplon,croplon:-croplon]

# COORDENADAS PARA POSICIONAR ESTACAO
if (estlat > np.amin(lats) and
    estlat < np.amax(lats) and
    estlon > np.amin(longs)and
    estlon < np.amax(longs)):                
    esty     = lats[:,0].tolist().index(takeClosest(lats[:,0],estlat))     #index Lat da estação no domínio
    estx     = longs[0,:].tolist().index(takeClosest(longs[0,:],estlon))   #index Lat da estação no domínio

# PLOT CORNERS
llcrnrlon = np.min(longs)      #longitude of lower left hand corner of the selected map domain.
llcrnrlat = np.min(lats)       #latitude of upper right hand corner of the desired map domain (degrees).
urcrnrlon = np.max(longs)      #longitude of upper right hand corner of the selected map domain.
urcrnrlat = np.max(lats)      #latitude of upper right hand corner of the selected map domain.

coordlat  = np.linspace(llcrnrlat,urcrnrlat,num=aconc[0].dims['ROW'])
coordlon  = np.linspace(llcrnrlon,urcrnrlon,num=aconc[0].dims['COL'])

# ADJUSTMENTS
for a in aconc:
    a.coords['TSTEP'] = dates
    a.coords['ROW'] = coordlat
    a.coords['COL'] = coordlon
    a.rename({'ROW': 'LAT', 'COL': 'LON'}, inplace=True)
    for var in a.variables:
        if 'units' in a.variables[var].attrs:
            if a.variables[var].attrs['units'].split() != []:
                a.variables[var].attrs['units'] = a.variables[var].attrs['units'].split()[0]

for i in range(0,len(aconc)):
    aconc[i] = aconc[i].isel(TSTEP=slice(spinup,spinup+run_len))
    aconc[i].coords['TSTEP'] -= pd.Timedelta(gmt,'h')
weekd=[]
for d in aconc[0].TSTEP.dt.weekday:
    if d < 5:
        weekd.append('Weekday')
    else:
        weekd.append('Weekend')

D:\Users\50033096\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: The inplace argument has been deprecated and will be removed in a future version of xarray.


## Cria DataFrames para Validação

In [46]:
pol = ['MP2_5','MP10','O3','NOx']
val_pm25  = pd.DataFrame(est[(aconc[0]['TSTEP'][0]):(aconc[0]['TSTEP'][-1])][pol[0]])
val_pm10  = pd.DataFrame(est[(aconc[0]['TSTEP'][0]):(aconc[0]['TSTEP'][-1])][pol[1]])
val_o3    = pd.DataFrame(est[(aconc[0]['TSTEP'][0]):(aconc[0]['TSTEP'][-1])][pol[2]])
val_nox   = pd.DataFrame(est[(aconc[0]['TSTEP'][0]):(aconc[0]['TSTEP'][-1])][pol[3]])

val = [val_pm25, val_pm10, val_o3, val_nox]
# MODELAGENS 
# Buscar dados das modelagens, usando mod_param
for i in range(0,len(aconc)):
# pm25 = PM25_TOT        ,ug/m3
    val[0][modnames[i]] = pd.Series(aconc[i].variables['PM25_TOT'][:,esty,estx], index=val_pm25.index)
# pm10 = PM10            ,ug/m3 
    val[1][modnames[i]] = pd.Series(aconc[i].variables['PM10'][:,esty,estx], index=val_pm10.index)
# o3 = O3_UGM3        ,ug/m3
    val[2][modnames[i]] = pd.Series(aconc[i].variables['O3_UGM3'][:,esty,estx], index=val_o3.index)
# nox = NOX_UGM3        ,ug/m3
    val[3][modnames[i]] = pd.Series(aconc[i].variables['NOX_UGM3'][:,esty,estx], index=val_nox.index)

## Funções para validação

In [47]:
#RootMeanSquareError - RMSE
def meansqError(lmodel,lobs):
    sqerr = 0
    n = 0
    for i in range(0,len(lmodel)):
        if lobs[i] is not None:
            sqerr += ((lmodel[i]-lobs[i])**2)
            n += 1
    if n != 0:
        sqerr /= n
        sqerr = sqerr**0.5
        return sqerr
    else:
        return None

#Index of Agreement - IOA
def indAgree (lmodel, lobs):
    ioa = 0
    num = 0
    den = 0
    n   = 0
    obsavg = 0 
    for i in range(0,len(lobs)):
        if lobs[i] is not None:
            obsavg += lobs[i]
            n += 1
    if n != 0: 
        obsavg /= n
    else:
        obsavg = None
    if obsavg != None:
        for i in range(0,len(lmodel)):
            if lobs[i] is not None:
               num += (lmodel[i]-lobs[i])**2
               den1 = np.absolute(lmodel[i]-obsavg)
               den2 = np.absolute(lobs[i]-obsavg)
               den += (den1+den2)**2
        ioa = 1-(num/den)
        return ioa
    else:
        return None

#MeanAbsoluteGrossError - MAGE
def grossError(lmodel,lobs):
    age = 0
    n = 0
    for i in range(0,len(lmodel)):
        if lobs[i] is not None:
            age += np.absolute(lmodel[i]-lobs[i])
            n += 1
    if n != 0:
        return age/n
    else:
        return None
        
#Coeficiente de correlação (r): 
def cor_r(lmodel, lobs):
    dic = {'model':lmodel,'obs':lobs}
    df = pd.DataFrame(dic)
    df.dropna(subset=['obs'],inplace=True)
    return np.corrcoef(df['model'],df['obs'])[0,1]
    #cov = np.cov(df['model'],df['obs'])[0,1]
    #stdobs = np.std(df['obs'])
    #stdmod = np.std(df['model'])
    #return cov/(stdobs*stdmod)

#MeanBias - MB
def meanBias(lmodel,lobs):
    bias = 0
    n = 0
    for i in range(0,len(lmodel)):
#        print (i)
#        print (lmodel[i])
#        print (lobs[i])
        if lobs[i] is not None:
            bias += (lmodel[i]-lobs[i])
            n += 1
    if n != 0:
        return bias/n
    else:
        return None    

#MeanError - ME
def meanError(lmodel,lobs):
    bias = 0
    n = 0
    for i in range(0,len(lmodel)):
#        print (i)
#        print (lmodel[i])
#        print (lobs[i])
        if lobs[i] is not None:
            bias += np.absolute(lmodel[i]-lobs[i])
            n += 1
    if n != 0:
        return bias/n
    else:
        return None  
    

#Normalized Mean Bias – NMB
def normMeanBias(lmodel, lobs):
    bias = 0
    n = 0
    soma = 0
    for i in range(0,len(lmodel)):
        if lobs[i] is not None:
            bias += (lmodel[i]-lobs[i])
            soma += lobs[i]
            n += 1            
    if n != 0:
        return (bias/soma)
    else:
        return None    

#Normalized Mean Error – NME
def normMeanError(lmodel, lobs):
    err = 0
    n = 0
    soma = 0
    for i in range(0,len(lmodel)):
        if lobs[i] is not None:
            err += np.absolute(lmodel[i]-lobs[i])
            soma += lobs[i]
            n += 1            
    if n != 0:
        return (err/soma)
    else:
        return None 

#Mean Fractional Bias – MFB
def meanFracBias(lmodel, lobs):
    fracbias = 0
    n = 0
    soma = 0
    for i in range(0,len(lmodel)):
        if lobs[i] is not None:
            fracbias += ( (lmodel[i]-lobs[i]) / ((lmodel[i]+lobs[i]) / 2))
            n += 1            
    if n != 0:
        return fracbias/n
    else:
        return None   

#Mean Fractional Error – MFE
def meanFracError(lmodel, lobs):
    fracerr = 0
    n = 0
    soma = 0
    for i in range(0,len(lmodel)):
        if lobs[i] is not None:
            fracerr += ( np.absolute(lmodel[i]-lobs[i]) / ((lmodel[i]+lobs[i]) / 2))
            n += 1            
    if n != 0:
        return fracerr/n
    else:
        return None 
    
#Mean Fractional Bias Goal - MFBGoal
def mfbGoal(lmodel, lobs):
    print (np.average(lobs))
    print (np.average(lmodel))
    return 170 * np.exp(-0.5 * ( np.average(lobs) + np.average(lmodel) ) / 0.5) + 30 

#Mean Fractional Error Goal - MFEGoal
def mfeGoal(lmodel, lobs):
    return 150 * np.exp(-0.5 * ( np.average(lobs) + np.average(lmodel) ) / 0.75) + 50

#Mean Fractional Bias Criteria - MFBCriteria
def mfbCriteria(lmodel, lobs):
    return 140 * np.exp(-0.5 * ( np.average(lobs) + np.average(lmodel) ) / 0.5) + 60

#Mean Fractional Error Criteria - MFECriteria
def mfeCriteria(lmodel, lobs):
    return 125 * np.exp(-0.5 * ( np.average(lobs) + np.average(lmodel) ) / 0.75) + 75
   

## Validação

In [48]:
#DROPNAS
for i in range(0,len(val)):
    val[i].dropna(subset=[pol[i]],inplace=True)

In [49]:
cols = ['Poluentes','Mes','Mod',#'MB',
        'RMSE',#'MAGE','IOA',
        'NMB','NME',
        'MFB','MFBGoal','MFBCriteria',
        'MFE', 'MFEGoal', 'MFECriteria',
        'r']
cpoluentes = []
cmeses = []
cmodelos = []
#cMB   = []
cRMSE = []
#cMAGE = []
#cIOA  = []
cNMB  = []
cNME  = []

cMFB  = []
cMFBgoal = []
cMFBcriteria = []

cMFE  = []
cMFEgoal = []
cMFEcriteria =[]

cr    = []

for i in range(0,len(pol)):
    for m in modnames:
        cpoluentes.append(pol[i])
        cmeses.append(mes[date_index])
        cmodelos.append(m)
        #lmodel,lobs
        #cMB.append(   meanBias      (val[i][m],val[i][pol[i]])) 
        cRMSE.append( meansqError   (val[i][m],val[i][pol[i]]))
        #cMAGE.append( grossError    (val[i][m],val[i][pol[i]]))
        #cIOA.append(  indAgree      (val[i][m],val[i][pol[i]]))
        cNMB.append(  normMeanBias  (val[i][m],val[i][pol[i]])) 
        cNME.append(  normMeanError (val[i][m],val[i][pol[i]])) 
        
        cMFB.append(  meanFracBias  (val[i][m],val[i][pol[i]])) 
        cMFBgoal.append( mfbGoal (val[i][m],val[i][pol[i]]))
        cMFBcriteria.append( mfbCriteria (val[i][m],val[i][pol[i]]))
        
        cMFE.append(  meanFracError (val[i][m],val[i][pol[i]])) 
        cMFEgoal.append( mfeGoal (val[i][m],val[i][pol[i]]))
        cMFEcriteria.append( mfeCriteria (val[i][m],val[i][pol[i]]))
        
        cr.append(    cor_r         (val[i][m],val[i][pol[i]]))

d = {cols[0]:cpoluentes,
    cols[1]:cmeses,
    cols[2]:cmodelos,
    #cols[]:cMB,
    cols[3]:cRMSE,
    #cols[]:cMAGE,
    #cols[]:cIOA,
    cols[4]:cNMB,
    cols[5]:cNME,
    cols[6]:cMFB,
    cols[7]:cMFBgoal,
    cols[8]:cMFBcriteria,
    cols[9]:cMFE,
    cols[10]:cMFEgoal,
    cols[11]:cMFEcriteria,
    cols[12]:cr}

validf = pd.DataFrame(d)

16.732142857142858
2.7138033
16.732142857142858
29.422152
35.20238095238095
3.1717827
35.20238095238095
29.79194
103.0728476821192
66.16903
103.0728476821192
40.45242
8.108108108108109
0.99224347
8.108108108108109
92.73476


In [50]:
validf

,Poluentes,Mes,Mod,RMSE,NMB,NME,MFB,MFBGoal,MFBCriteria,MFE,MFEGoal,MFECriteria,r
0,MP2_5,jan,ED,16.038601,-0.837809,0.842305,-1.316301,30.000001,60.000001,1.359011,50.000351,75.000293,0.169144
1,MP2_5,jan,PT_ED-100I,43.664946,0.758421,1.800284,-0.281106,30.000000,60.000000,1.274814,50.000000,75.000000,-0.153749
2,MP10,jan,ED,34.557808,-0.909899,0.909899,-1.635130,30.000000,60.000000,1.635130,50.000000,75.000000,0.228066
3,MP10,jan,PT_ED-100I,43.420072,-0.153695,1.013347,-0.702809,30.000000,60.000000,1.229575,50.000000,75.000000,-0.102974
4,O3,jan,ED,56.766913,-0.358036,0.450181,-0.336958,30.000000,60.000000,0.504406,50.000000,75.000000,0.002603
5,O3,jan,PT_ED-100I,86.102992,-0.607536,0.669261,-0.827919,30.000000,60.000000,0.941536,50.000000,75.000000,-0.435027
6,NOx,jan,ED,9.816419,-0.877623,0.908205,-1.162409,30.018977,60.015628,1.475593,50.347752,75.289793,-0.082429
7,NOx,jan,PT_ED-100I,150.708360,10.437288,11.412601,0.222385,30.000000,60.000000,1.646892,50.000000,75.000000,-0.193325


In [15]:
modnames

['ED', 'PT_ED-100I']

In [16]:
pol

['MP2_5', 'MP10', 'O3', 'NOx']

In [23]:
type(val)

list

In [24]:
len(val)

4

In [60]:
np.corrcoef(val[2][pol[2]], val[2][modnames[0]])[0,1]

0.00260305442609114

In [58]:
np.corrcoef(val[2]['O3'], val[2]['PT_ED-100I'])[0,1]

-0.4350266386727775

In [53]:
val[2][pol[2]]

,O3,ED,PT_ED-100I
Data,,,
2015-01-11 07:00:00,36.0,66.898804,66.825134
2015-01-11 08:00:00,37.0,66.502579,33.704323
2015-01-11 09:00:00,49.0,71.291405,12.806714
2015-01-11 10:00:00,75.0,66.554436,13.000618
2015-01-11 11:00:00,110.0,60.916656,15.497415
2015-01-11 12:00:00,130.0,58.404469,6.338462
2015-01-11 13:00:00,139.0,58.988152,4.399853
2015-01-11 14:00:00,132.0,60.425591,15.050270
2015-01-11 15:00:00,127.0,62.350060,39.153809


In [54]:
val[2][pol[2]]

Data
2015-01-11 07:00:00     36.0
2015-01-11 08:00:00     37.0
2015-01-11 09:00:00     49.0
2015-01-11 10:00:00     75.0
2015-01-11 11:00:00    110.0
2015-01-11 12:00:00    130.0
2015-01-11 13:00:00    139.0
2015-01-11 14:00:00    132.0
2015-01-11 15:00:00    127.0
2015-01-11 16:00:00    125.0
2015-01-11 17:00:00    123.0
2015-01-11 18:00:00    108.0
2015-01-11 19:00:00    103.0
2015-01-11 20:00:00     84.0
2015-01-11 21:00:00     81.0
2015-01-11 22:00:00     67.0
2015-01-11 23:00:00     64.0
2015-01-12 00:00:00     66.0
2015-01-12 01:00:00     58.0
2015-01-12 02:00:00     76.0
2015-01-12 03:00:00     67.0
2015-01-12 04:00:00     81.0
2015-01-12 05:00:00     99.0
2015-01-12 07:00:00    103.0
2015-01-12 08:00:00    113.0
2015-01-12 09:00:00    102.0
2015-01-12 10:00:00    109.0
2015-01-12 11:00:00    128.0
2015-01-12 12:00:00    136.0
2015-01-12 13:00:00    155.0
                       ...  
2015-01-16 14:00:00    136.0
2015-01-16 15:00:00    136.0
2015-01-16 16:00:00    139.0
2015-01-1

In [55]:
val[2][modnames[1]]

Data
2015-01-11 07:00:00    66.825134
2015-01-11 08:00:00    33.704323
2015-01-11 09:00:00    12.806714
2015-01-11 10:00:00    13.000618
2015-01-11 11:00:00    15.497415
2015-01-11 12:00:00     6.338462
2015-01-11 13:00:00     4.399853
2015-01-11 14:00:00    15.050270
2015-01-11 15:00:00    39.153809
2015-01-11 16:00:00    21.979267
2015-01-11 17:00:00     4.280704
2015-01-11 18:00:00     5.623199
2015-01-11 19:00:00    37.650108
2015-01-11 20:00:00    56.388103
2015-01-11 21:00:00    63.726364
2015-01-11 22:00:00    63.095402
2015-01-11 23:00:00    62.322430
2015-01-12 00:00:00    62.044731
2015-01-12 01:00:00    62.210403
2015-01-12 02:00:00    62.313747
2015-01-12 03:00:00    61.992752
2015-01-12 04:00:00    62.238091
2015-01-12 05:00:00    62.843124
2015-01-12 07:00:00    62.886642
2015-01-12 08:00:00    29.633432
2015-01-12 09:00:00     8.964066
2015-01-12 10:00:00     8.762889
2015-01-12 11:00:00     8.989385
2015-01-12 12:00:00    13.807670
2015-01-12 13:00:00    14.120878
     